# SAVE PICKLES?


In [11]:
SAVING = True

In [3]:
SAVING = False

# IMPORTING DATA

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
import pandas as pd
from pathlib import Path

# Local imports
#------------------------------------------------
import WireDAQ.PandasPlus           # Make sure this import is after pandas
import WireDAQ.Constants as cst
import WireDAQ.NXCALS as nx
import WireDAQ.Parser as parser
# import WireDAQ.Efficiency as eff



# Creating NXCALS variable containers
#------------------------------------------------
wires     = {'B1': [nx.NXCALSWire(loc = loc) for loc in ['L1B1','L5B1']],
             'B2': [nx.NXCALSWire(loc = loc) for loc in ['R1B2','R5B2']]}
beams     = [nx.NXCALSBeam(name) for name in ['B1','B2']]
LHC       = nx.NXCALSLHC()
b_slots   = np.arange(3564)
#------------------------------------------------

# Locating data
# _default_path   = '/home/phbelang/002mount/'
_default_path   = '/eos/project/l/lhc-lumimod/LuminosityFollowUp/2023/rawdata/'


# Loading wire current for each fill
# variables = ['HX:FILLN','HX:BMODE','HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Intensity for beam in beams]
variables = ['HX:FILLN','HX:BMODE','HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Nb for beam in beams]
file_list = list(Path(_default_path).glob('*/HX:BMODE=STABLE/*.parquet'))




to_exclude = ['8782','8817','8821','8675']
file_list = [f for f in file_list if str(f).split('FILLN=')[1][:4] not in to_exclude ]

df = parser.from_parquet(file=file_list, variables=variables)


# Filtering: wires on  

In [12]:

for _var in ['HX:BMODE','HX:FILLN',"HX:BETASTAR_IP1",beams[0].Nb,beams[1].Nb]:
    df[_var] = df[_var].ffill()

filtered = df.loc[  (df["HX:BMODE"] == 'STABLE') & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Nb] > 200)&
                    (df[beams[1].Nb] > 200)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Nb for beam in beams]].max().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in  wires['B2']]
relevant_idx    = results.index[(np.any(results[current_culumns]>50,axis=1))]

results = results.loc[relevant_idx]
results.index = results.index.astype(int)
results.sort_index(inplace=True)

display(results)
print(f'{len(results)} fills found')

if SAVING:
    results.to_pickle("filter_wires_on.pkl")

,HX:BETASTAR_IP1,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BQM.B1:NO_BUNCHES,LHC.BQM.B2:NO_BUNCHES
HX:FILLN,,,,,
8686,30.0,350.019196,350.041412,399.0,399.0
8690,30.0,350.020203,350.039276,399.0,399.0
8695,30.0,350.017487,350.046051,911.0,911.0
8696,30.0,350.020782,350.043060,999.0,999.0
8701,30.0,350.014587,350.043091,999.0,999.0
8723,30.0,350.018433,350.042236,399.0,399.0
8725,30.0,350.013550,350.037964,999.0,999.0
8729,30.0,350.015137,350.041656,1163.0,1163.0
8730,30.0,350.020233,350.051941,1163.0,1163.0


21 fills found


# Filtering: wires off

In [13]:

for _var in ['HX:BMODE','HX:FILLN',"HX:BETASTAR_IP1",beams[0].Nb,beams[1].Nb]:
    df[_var] = df[_var].ffill()

filtered = df.loc[  (df["HX:BMODE"] == 'STABLE') & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Nb] > 200)&
                    (df[beams[1].Nb] > 200)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Nb for beam in beams]].max().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in wires['B2']]
relevant_idx    = results.index[np.invert(np.any(results[current_culumns]>50,axis=1))]

results = results.loc[relevant_idx]
results.index = results.index.astype(int)
results.sort_index(inplace=True)

display(results)
print(f'{len(results)} fills found')

if SAVING:
    results.to_pickle("filter_wires_off.pkl")

,HX:BETASTAR_IP1,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BQM.B1:NO_BUNCHES,LHC.BQM.B2:NO_BUNCHES
HX:FILLN,,,,,
8685,30.0,0.016420,0.039100,411.0,411.0
8731,30.0,0.012387,0.041009,1818.0,1818.0
8741,30.0,0.014297,0.040100,1903.0,1903.0
8804,30.0,0.013705,0.033221,2374.0,2374.0
8850,30.0,0.012052,0.033797,705.0,705.0
8853,30.0,0.017679,0.036160,1195.0,1195.0
8858,30.0,0.013933,0.035676,1650.0,1650.0
8863,30.0,0.014419,0.035827,1650.0,1650.0
8870,30.0,0.015420,0.036691,1886.0,1886.0


9 fills found


# Quality check: negative current?



In [5]:

for _var in ['HX:BMODE','HX:FILLN',"HX:BETASTAR_IP1",beams[0].Intensity,beams[1].Intensity]:
    df[_var] = df[_var].ffill()

filtered = df.loc[  (df["HX:BMODE"] == 'STABLE') &
                    (df['HX:FILLN'].astype(float) >= 8033) & 
                    (df["HX:BETASTAR_IP1"] == 30)&
                    (df[beams[0].Intensity] > 300*1e11)&
                    (df[beams[1].Intensity] > 300*1e11)]

# Looking at values of max current
results = filtered.groupby('HX:FILLN')[['HX:BETASTAR_IP1'] + [wire.I for wire in wires['B2']] + [beam.Intensity for beam in beams]].max().dropna()

# Keeping only wires off
current_culumns = [wire.I for wire in wires['B2']]
relevant_idx    = results.index[(np.any(results[current_culumns]<-10,axis=1))]

results = results.loc[relevant_idx]
results.index = results.index.astype(int)
results.sort_index(inplace=True)

display(results)
print(f'{len(results)} fills found')


,HX:BETASTAR_IP1,RPMC.UL16.RBBCW.R1B2:I_MEAS,RPMC.UL557.RBBCW.R5B2:I_MEAS,LHC.BCTDC.A6R4.B1:BEAM_INTENSITY,LHC.BCTDC.A6R4.B2:BEAM_INTENSITY
HX:FILLN,,,,,


0 fills found
